In [1]:
!pip install vtk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 MB 8.4 MB/s eta 0:00:00


In [32]:
import vtk
import numpy as np

In [33]:
def load_vector_field(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    return reader.GetOutput(), reader.GetOutput().GetBounds()

In [34]:
def rk4_integration(start_point, step_size, max_steps):
    current_point = np.array(start_point)
    points = vtk.vtkPoints()
    points.InsertNextPoint(current_point)

    data, bounds = load_vector_field("tornado3d_vector.vti")

    for _ in range(max_steps):
        if not all(bounds[i] <= current_point[i//2] <= bounds[i+1] for i in range(0, len(bounds), 2)):
            break  # Stop if any coordinate of the current point is out of bounds



        vec = probe_vector(current_point, data)

        k1 = np.array(vec)
        k2 = np.array(probe_vector((current_point + k1 * (step_size / 2))[0], data))
        k3 = np.array(probe_vector((current_point + k2 * (step_size / 2))[0], data))
        k4 = np.array(probe_vector((current_point + k3 * step_size)[0], data))

        next_point = current_point + step_size * ((k1 + 2*k2 + 2*k3 + k4) / 6.0)
        points.InsertNextPoint(current_point)
        current_point = next_point[0]


    return points

In [35]:
def probe_vector(point, data):
    # Create a locator to find the closest point
    points = vtk.vtkPoints()
    points.InsertNextPoint(point)

    points_poly = vtk.vtkPolyData()
    points_poly.SetPoints(points)

    probe = vtk.vtkProbeFilter()
    probe.SetSourceData(data)
    probe.SetInputData(points_poly)
    probe.Update()

    vector_array = probe.GetOutput().GetPointData().GetArray("vectors")

    # Convert vector_array to a numpy array for easier manipulation
    num_tuples = vector_array.GetNumberOfTuples()
    vector_size = vector_array.GetNumberOfComponents()
    vectors = np.zeros((num_tuples, vector_size))
    for i in range(num_tuples):
        vectors[i] = vector_array.GetTuple(i)

    return vectors

In [36]:
def module_write_to_file(filename,polydata):
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(polydata)
    writer.Write()


def write_vtp(filename, points):
    # Create polyline
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(points.GetNumberOfPoints())
    for i in range(points.GetNumberOfPoints()):
        polyline.GetPointIds().SetId(i, i)

    # Create polyline cells
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)

    # Create polydata
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetLines(cells)

    module_write_to_file(filename,polydata)

    # Write to file
#     writer = vtk.vtkXMLPolyDataWriter()
#     writer.SetFileName(filename)
#     writer.SetInputData(polydata)
#     writer.Write()

In [43]:
def main():
    # User input for seed location
    # seed_location = [0.0, 0.0, 7.0]
    x = int(input("Enter the x-coordinate for the seed: "))
    y = int(input("Enter the y-coordinate for the seed: "))
    z = int(input("Enter the z-coordinate for the seed: "))

    # Initialize seed_location with user inputs
    seed_location = [x, y, z]
    # print(seed_location)

    # Integration parameters
    step_size = 0.05
    max_steps = 1000

    # Perform RK4 integration forward and backward
    interpolate_ahead = rk4_integration(seed_location, step_size, max_steps)
    interpolate_back = rk4_integration(seed_location, -step_size, max_steps)

    # Combine forward and backward points
    allpoints = vtk.vtkPoints()

    for i in range(interpolate_back.GetNumberOfPoints() - 1, -1, -1):
        allpoints.InsertNextPoint(interpolate_back.GetPoint(i))

    allpoints.InsertNextPoint(seed_location)

    for i in range(interpolate_ahead.GetNumberOfPoints()):
        allpoints.InsertNextPoint(interpolate_ahead.GetPoint(i))

    # Write combined points to VTP file
    write_vtp("streamline.vtp", allpoints)
    print("generated")
    # print(allpoints)


if __name__ == "__main__":
    main()

Enter the x-coordinate for the seed: 0
Enter the y-coordinate for the seed: 0
Enter the z-coordinate for the seed: 7
generated
